In [ ]:
!pip install gurobipy
import pandas
import numpy as np
import pandas as pd
from gurobipy import*

In [ ]:
df = pd.read_excel('Midterm-Data (1).xlsx',index_col=0)
nodes = range(df.shape[0])

dist = [[df[j][i] for j in nodes] for i in nodes]

k = 3
veh_cap = 150


In [ ]:

 data = pandas.read_csv("demand.csv")
demand=data

In [ ]:

  demand = data.values.tolist()
  demand

[[0.0, nan],
 [13.0, nan],
 [2.0, 95.0],
 [3.0, 54.0],
 [4.0, 61.0],
 [5.0, 3.0],
 [6.0, 22.0],
 [7.0, 15.0],
 [8.0, 38.0],
 [9.0, 42.0],
 [10.0, 34.0]]

In [ ]:
import gurobipy as gp

In [ ]:
vrp = Model("vrp_model")

Restricted license - for non-production use only - expires 2025-11-24


In [ ]:
x = vrp.addVars(nodes, nodes, lb = 0,vtype = GRB.BINARY,name = 'x')
u = vrp.addVars(nodes, lb = 0,vtype = GRB.INTEGER,name = 'u')

In [ ]:
vrp.addConstrs(((quicksum(x[i,j] for j in nodes if j != i) == 1) for i in nodes if i != 0), name = '1');
vrp.addConstrs(((quicksum(x[i,j] for i in nodes if i != j) == 1) for j in nodes if j != 0), name = '2');
vrp.addConstr(((quicksum(x[0,j] for j in nodes) == k)), name = '3');
vrp.addConstr(((quicksum(x[i,0] for i in nodes) == k)), name = '4');
vrp.addConstrs(((u[i] <= veh_cap) for i in nodes), name = '6');
vrp.addConstrs(((u[i] >= demand[i]) for i in nodes), name = '7');


KeyError: 0

In [ ]:
vrp.setObjective((quicksum(dist[i][j]*x[i,j] for i in nodes for j in nodes if i != j)),GRB.MINIMIZE)
vrp.setParam("TimeLimit", 120)


Set parameter TimeLimit to value 120


In [ ]:
vrp.update()
vrp.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 218 rows, 132 columns and 1998 nonzeros
Model fingerprint: 0xd04a2d8a
Variable types: 0 continuous, 132 integer (121 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 2196.5000000
Presolve removed 196 rows and 21 columns
Presolve time: 0.00s
Presolved: 22 rows, 111 columns, 222 nonzeros
Variable types: 0 continuous, 111 integer (111 binary)

Root relaxation: objective 1.438400e+03, 23 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0

In [ ]:
status = vrp.status
object_Value = vrp.objVal
print()
print("model status is: ", status)
print()
print("Objective value is: ", object_Value)


model status is:  2

Objective value is:  1438.4


In [ ]:
if status !=3 and status != 4:
    for v in vrp.getVars():
        if vrp.objVal < 1e+99  and v.x!=0:
            print('%s %f'%(v.Varname,v.x))


x[0,0] 1.000000
x[0,4] 1.000000
x[0,9] 1.000000
x[1,2] 1.000000
x[2,1] 1.000000
x[3,10] 1.000000
x[4,0] 1.000000
x[5,8] 1.000000
x[6,7] 1.000000
x[7,5] 1.000000
x[8,6] 1.000000
x[9,0] 1.000000
x[10,3] 1.000000


In [ ]:
  if status != 3 and status != 4:
    vis = []
    Sol_x = np.zeros([len(nodes), len(nodes)])
    for i in nodes:
        for j in nodes:
            if vrp.objVal < 1e+99:
                Sol_x[i,j] = x[i,j].getAttr("X")
            else:
                error_status = True
                ofvv = 1e+99
            if 1 - 0.00001 <= Sol_x[i,j] <= 1 + 0.00001:
                vis.append((i,j))

visited = np.array(vis)
solution = []
if visited[0][0] == 0:
    sol = [visited[0][0], visited[0][1]]
elif visited[0][0] !=0 and visited[0][1] == 0:
    sol = [visited[0][1], visited[0][0]]
else:
    print('First tuple should include depot 0')
visited = np.delete(visited,0, axis = 0)

solution = []
for i in visited:
    try:
        next_ind = int(np.where(visited[:,0] == sol[-1])[0])
        sol.append(visited[next_ind][1])
        visited = np.delete(visited,next_ind, axis = 0)
    except:
        next_ind = int(np.where(visited[:,1] == sol[-1])[0])
        sol.append(visited[next_ind][0])
        visited = np.delete(visited,next_ind, axis = 0)

    if sol[0] == sol[-1]:
        sol = np.asarray(sol)
        solution.append(sol)
        used = []
        for j in solution:
            for k in j:
                used.append(k)
        remain = list(set(nodes) - set(used))
        if remain == []:
            break
        sol=[visited[0][0], visited[0][1]]
        visited = np.delete(visited,0, axis = 0)

print('optimal Solution is:', solution)

TypeError: only length-1 arrays can be converted to Python scalars